---
layout: notes
chapter: 4
chapter-title: Training Models
permalink: /ml-book/chapter4/notes.html
---

## Training Models
This chapter is all about what's under the hood of models. Understanding this will help us:
* Pick the appropriate model
* Use the right training algorithm
* Adjust hyperparameters
* Help with debugging and error analysis
* Help with understanding neural networks when we get to them

### Training a Linear Regression model
* Using a direct "closed-form" equation that computer the model parameters based on the best fit of the model to the data
* Using an iterative optimization approach such as Gradient Descent (GD) which finds parameters by tweaking parameters and minimizing the cost function

### Other Regression Models
* We'll look at Polynomial Regression which is more complex model, but also prone to overfitting
* We'll look at Logistic Regression and Softmax Regression

## Linear Regression

$$ \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n $$

where:
* $\hat{y}$ is the predicted value
* $n$ is the number of features
* $x_i$ is the $i^{th}$ feature value
* $\theta_j$ is the $j^{th}$ model parameter ($\theta_0$ is the bias term and other $\theta_j$ terms are the feature weights)

#### Vectorized Form

$$ \hat{y} = h_{\theta}(\textbf{x}) = \theta^T \cdot \textbf{x} $$

Note here that $ x_0 = 1 $ to support the bias term $ \theta_0 $.

RMSE for Linear Regression:

$$ MSE(\textbf{X}, h_{\theta}) = \frac{1}{m} \sum_{i=1}^{m} (\theta^T \cdot \textbf{x}^{(i)} - y^{(i)})^2 $$

The summation is over all the samples in the data set. So i = 1 is the first sample of the dataset, i = 2 the second and so on, up to m.